In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import shutil
import time


In [2]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Filter images containing dogs (class 5 in CIFAR-10)
dog_indices_train = np.where(train_labels == 5)[0]
dog_indices_test = np.where(test_labels == 5)[0]

# Create new labels: 1 for dogs, 0 for everything else
train_labels_binary = np.zeros((len(train_labels), 1))
train_labels_binary[dog_indices_train] = 1

test_labels_binary = np.zeros((len(test_labels), 1))
test_labels_binary[dog_indices_test] = 1

# Filter images containing dogs and non-dog images
train_images_dog = train_images[dog_indices_train]
train_images_nondog = np.delete(train_images, dog_indices_train, axis=0)

test_images_dog = test_images[dog_indices_test]
test_images_nondog = np.delete(test_images, dog_indices_test, axis=0)

# Combine dog and non-dog images
train_images_final = np.concatenate((train_images_dog, train_images_nondog))
test_images_final = np.concatenate((test_images_dog, test_images_nondog))

# Combine dog and non-dog labels
train_labels_final = np.concatenate((train_labels_binary[dog_indices_train], np.zeros((len(train_images_nondog), 1))))
test_labels_final = np.concatenate((test_labels_binary[dog_indices_test], np.zeros((len(test_images_nondog), 1))))

# Determine the number of non-dog images to delete
num_images_to_delete = 35000

# Delete the specified number of non-dog images
train_images_nondog_balanced = np.delete(train_images_nondog, range(num_images_to_delete), axis=0)

# Update the combined train images and labels
train_images_final = np.concatenate((train_images_dog, train_images_nondog_balanced))
train_labels_final = np.concatenate((train_labels_binary[dog_indices_train], np.zeros((len(train_images_nondog_balanced), 1))))

# Create an ImageDataGenerator instance for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your training data
datagen.fit(train_images_final)

In [3]:
# Define the model architecture

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 30, 30, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                        

In [4]:
# Define a function to perform inference on your model
def perform_inference(model, input_data):
    # Record start time
    start_time = time.time()
    
    # Run inference
    model.predict(input_data)
    
    # Record end time
    end_time = time.time()
    
    # Calculate inference time
    inference_time = end_time - start_time
    
    return inference_time

# Example: Perform inference and measure inference rate
num_inferences = 100  # Number of inferences to perform

# Placeholder input data (replace this with your actual input data)
input_shape = (32, 32, 3)  # Specify the shape of your input data
input_data = np.random.rand(num_inferences, *input_shape)  # Generate random input data

# Perform inferences and measure total inference time
total_inference_time = 0
for _ in range(num_inferences):
    total_inference_time += perform_inference(model, input_data)

# Calculate average inference time
average_inference_time = total_inference_time / num_inferences

# Calculate inference rate (inferences per second)
inference_rate = num_inferences / total_inference_time

print("Average Inference Time:", average_inference_time, "seconds")
print("Inference Rate:", inference_rate, "inferences/second")



4/4 [==============================] - 0s 3ms/step
Average Inference Time: 0.10513855934143067 seconds
Inference Rate: 9.511258345785059 inferences/second
